<h1>Gain Life Challenge</h1>

<h2>Import Dataset</h2>

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
df = pd.read_csv(r"C:\Users\justi\Downloads\Anonymized Message Data.csv")
df

,thread_id,timestamp,role,content
0,0,6/7/2024 14:47,supervisor,"Good morning, Ms. Cooper - when you have a few..."
1,0,6/10/2024 13:18,claimant,1. What is your job title/department? Animal C...
2,0,6/10/2024 13:39,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...
3,0,7/1/2024 19:03,supervisor,Hi Sarah - According to the documents from you...
4,0,7/2/2024 3:35,claimant,I was unable to make it to an appointment on 6...
...,...,...,...,...
3074,946,7/1/2022 13:39,claimant,No thanks
3075,947,10/4/2022 13:30,claimant,"Sarah, if you recall in the documentation ther..."
3076,948,9/5/2024 13:56,claimant,"Hello, Ms.Smith I sent you & sign the paperwor..."
3077,948,9/13/2024 12:32,claimant,"Ms. Smith, I went back to work on September 3r..."


<h2>Data Cleaning & Formatting</h2>

In [63]:
df = df.drop_duplicates()

In [65]:
df = df.dropna()

<p>Drop duplicates while disregarding the seconds from the timestamp column. Noticed duplicate messages were sent a couple seconds after each other. Assuming those messages were mistakes and have been dropped (8 rows affected)</p>

In [68]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp_truncated'] = df['timestamp'].dt.floor('min')  # Remove seconds
df = df.drop_duplicates(subset=['thread_id', 'timestamp_truncated', 'role', 'content'], keep='first')
df = df.drop(columns=['timestamp_truncated'])  # Remove helper column
df

,thread_id,timestamp,role,content
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few..."
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...
...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther..."
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor..."
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r..."


<p>Get the rows that do not contain 'system' or 'admin' in the 'role' column (3 rows affected)</p>

In [71]:
df = df[~df["role"].str.contains("system|admin", case=False, na=False)]
df

,thread_id,timestamp,role,content
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few..."
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...
...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther..."
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor..."
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r..."


<p>Order the timestamps by ascending order (earliest first).</p>

In [74]:
df = df.sort_values(by=["thread_id", "timestamp"])

In [76]:
# import os

# # Define the path to save the file
# save_path = r"C:\Users\justi\Downloads\Gain Life Challenge"  # Replace with your desired folder

# # Save each DataFrame to the specified path
# df.to_csv(os.path.join(save_path, 'output2.csv'), index=True)

<h2>Data Analysis</h2>

<h3>Creating dimensions to set up relationships in PowerBI using star schema</h3>

In [80]:
roles_dimension = df['role'].drop_duplicates().reset_index(drop=True).reset_index()
roles_dimension = roles_dimension.rename(columns={'index': 'role_id'})
roles_dimension

,role_id,role
0,0,supervisor
1,1,claimant
2,2,adjuster
3,3,superAdjuster


In [105]:
date_dimension = pd.DataFrame(df['timestamp'].dt.date.unique(), columns=['date'])
date_dimension['date'] = pd.to_datetime(date_dimension['date'])
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['year'] = date_dimension['date'].dt.year
date_dimension

,date,day,month,year
0,2024-06-07,7,6,2024
1,2024-06-10,10,6,2024
2,2024-07-01,1,7,2024
3,2024-07-02,2,7,2024
4,2024-07-09,9,7,2024
...,...,...,...,...
679,2022-10-17,17,10,2022
680,2025-01-13,13,1,2025
681,2023-05-10,10,5,2023
682,2023-11-02,2,11,2023


<h3>Exploring how many threads were unanswered from each role</h3>

In [82]:
df['has_response'] = df.groupby('thread_id')['role'].transform('nunique') > 1
df

,thread_id,timestamp,role,content,has_response
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few...",True
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...,True
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,True
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...,True
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...,True
...,...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks,False
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther...",False
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",True
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r...",True


<h3>Calculate which role takes the longest to respond</h3>

<p>Used to find out what causes delays in the claims process</p>

In [86]:
# Calculate time between each message in a thread
df['next_message_time'] = df.groupby('thread_id')['timestamp'].shift(-1)
df['time_to_next_message'] = (df['next_message_time'] - df['timestamp']).dt.total_seconds() / 86400  # number of seconds in a day
df

,thread_id,timestamp,role,content,has_response,next_message_time,time_to_next_message
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few...",True,2024-06-10 13:18:00,2.938194
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...,True,2024-06-10 13:39:00,0.014583
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,True,2024-07-01 19:03:00,21.225000
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...,True,2024-07-02 03:35:00,0.355556
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...,True,2024-07-09 16:12:00,7.525694
...,...,...,...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks,False,NaT,NaN
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther...",False,NaT,NaN
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",True,2024-09-13 12:32:00,7.941667
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r...",True,2024-09-13 13:33:00,0.042361


<h3>Claim duration and initial response time correlation (Assuming it is the same topic in the threads)</h3>

In [89]:
# Calculate claim duration (time from first message to last)
df['duration_days'] = df.groupby('thread_id')['timestamp'].transform(lambda x: (x.max() - x.min()).days)
df

,thread_id,timestamp,role,content,has_response,next_message_time,time_to_next_message,duration_days
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few...",True,2024-06-10 13:18:00,2.938194,32
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...,True,2024-06-10 13:39:00,0.014583,32
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,True,2024-07-01 19:03:00,21.225000,32
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...,True,2024-07-02 03:35:00,0.355556,32
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...,True,2024-07-09 16:12:00,7.525694,32
...,...,...,...,...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks,False,NaT,NaN,0
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther...",False,NaT,NaN,0
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",True,2024-09-13 12:32:00,7.941667,7
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r...",True,2024-09-13 13:33:00,0.042361,7


<h3>Sentiment analysis on claimants</h3>

In [92]:

# Define sentiment keywords
sentiment_keywords = {
    'happy': ['thank', 'thanks', 'appreciate', 'great', 'good', 'excellent', 'perfect', 'got it', 
              'helpful', 'ty', 'ty!', 'thank you', 'wonderful', 'awesome'],
    
    'dissatisfied': ['wait', 'delay', 'sorry', 'unfortunately', 'problem', 'issue', 'concern', 
                    'worried', 'frustrated', 'annoyed', 'wrong', 'mistake', 'error',
                    'not', "haven't", "didn't", "don't", "can't", "unable", "fail", 
                    'missing', 'lost', 'confused', 'difficult', 'trouble', 'denied', 'reject'],
}

# Function to detect sentiment in a message
def detect_sentiment(text):
    text = text.lower()

    if any(keyword in text for keyword in sentiment_keywords['happy']):
        return 'happy'

    if any(keyword in text for keyword in sentiment_keywords['dissatisfied']):
        return 'dissatisfied'

    return 'neutral'

# Apply sentiment detection to claimant messages only
df['sentiment'] = 'neutral'  # Default value
claimant_filter = df['role'] == 'claimant' #Get only claimant roles
df.loc[claimant_filter, 'sentiment'] = df.loc[claimant_filter, 'content'].apply(detect_sentiment)
df

,thread_id,timestamp,role,content,has_response,next_message_time,time_to_next_message,duration_days,sentiment
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few...",True,2024-06-10 13:18:00,2.938194,32,neutral
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...,True,2024-06-10 13:39:00,0.014583,32,happy
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,True,2024-07-01 19:03:00,21.225000,32,neutral
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...,True,2024-07-02 03:35:00,0.355556,32,neutral
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...,True,2024-07-09 16:12:00,7.525694,32,dissatisfied
...,...,...,...,...,...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks,False,NaT,NaN,0,happy
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther...",False,NaT,NaN,0,happy
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",True,2024-09-13 12:32:00,7.941667,7,happy
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r...",True,2024-09-13 13:33:00,0.042361,7,happy


<h3>Explore how many new, existing and closed </h3>

In [101]:
# Extract year and month
df['year_month'] = df['timestamp'].dt.to_period('M')

# Find first and last message for each thread
df['start_date'] = df.groupby('thread_id')['timestamp'].transform('min').dt.to_period('M')
df['end_date'] = df.groupby('thread_id')['timestamp'].transform('max').dt.to_period('M')

df

,thread_id,timestamp,role,content,has_response,next_message_time,time_to_next_message,duration_days,sentiment,year_month,start_date,end_date
0,0,2024-06-07 14:47:00,supervisor,"Good morning, Ms. Cooper - when you have a few...",True,2024-06-10 13:18:00,2.938194,32,neutral,2024-06,2024-06,2024-07
1,0,2024-06-10 13:18:00,claimant,1. What is your job title/department? Animal C...,True,2024-06-10 13:39:00,0.014583,32,happy,2024-06,2024-06,2024-07
2,0,2024-06-10 13:39:00,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,True,2024-07-01 19:03:00,21.225000,32,neutral,2024-06,2024-06,2024-07
3,0,2024-07-01 19:03:00,supervisor,Hi Sarah - According to the documents from you...,True,2024-07-02 03:35:00,0.355556,32,neutral,2024-07,2024-06,2024-07
4,0,2024-07-02 03:35:00,claimant,I was unable to make it to an appointment on 6...,True,2024-07-09 16:12:00,7.525694,32,dissatisfied,2024-07,2024-06,2024-07
...,...,...,...,...,...,...,...,...,...,...,...,...
3074,946,2022-07-01 13:39:00,claimant,No thanks,False,NaT,NaN,0,happy,2022-07,2022-07,2022-07
3075,947,2022-10-04 13:30:00,claimant,"Sarah, if you recall in the documentation ther...",False,NaT,NaN,0,happy,2022-10,2022-10,2022-10
3076,948,2024-09-05 13:56:00,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",True,2024-09-13 12:32:00,7.941667,7,happy,2024-09,2024-09,2024-09
3077,948,2024-09-13 12:32:00,claimant,"Ms. Smith, I went back to work on September 3r...",True,2024-09-13 13:33:00,0.042361,7,happy,2024-09,2024-09,2024-09


In [109]:
# import os

# # Define the path to save the file
# save_path = r"C:\Users\justi\Downloads\Gain Life Challenge"  # Replace with your desired folder

# # Save each DataFrame to the specified path
# df.to_csv(os.path.join(save_path, 'facts.csv'), index=True)
# roles_dimension.to_csv(os.path.join(save_path, 'roles_dimension.csv'), index=True)
# date_dimension.to_csv(os.path.join(save_path, 'date_dimension.csv'), index=True)